In [1]:
import pickle
import numpy as np
import pandas as pd

# Profiles

In [2]:
with open('./save_feat/feature_adaface_ir101_webface12m_profiles.pickle', 'rb') as f:
    profiles_feats = pickle.load(f)

In [3]:
len(profiles_feats.keys())

4443

In [4]:
np.array(list(profiles_feats.values())).squeeze(1).shape

(4443, 512)

In [5]:
profiles_face_ids = []
profiles_image_names = []
for k in profiles_feats.keys():
    profiles_face_ids.append(k.split('_')[0])
    profiles_image_names.append(k)


In [6]:
profiles_df = pd.DataFrame({'id':profiles_face_ids, 'feat':list(profiles_feats.values()), 'image_name':profiles_image_names})

In [7]:
profiles_df

,id,feat,image_name
0,4470,"[[-0.022982748, 0.021464614, 0.01820613, 0.086...",4470_3.jpg
1,1737,"[[0.036668584, 0.036190886, -0.04464412, -0.02...",1737_2.jpg
2,390,"[[-0.07940415, 0.052058954, 0.005915749, -0.00...",390_11.jpg
3,4985,"[[-0.042736806, 0.055953614, -0.02027406, 0.00...",4985_1.jpg
4,1667,"[[0.04142807, 0.07902482, 0.07626078, -0.03072...",1667_2.jpg
...,...,...,...
4438,961,"[[-0.057810713, -0.09719874, -0.052204117, -0....",961_6.jpg
4439,641,"[[0.04617567, 0.031611796, -0.036156558, -0.13...",641_11.jpg
4440,3041,"[[0.039365545, 0.091820374, -0.009164525, 0.03...",3041_1.jpg
4441,4870,"[[-0.1046285, 0.019004075, -0.073794365, 0.017...",4870_2.jpg


In [8]:
profiles_mat = np.array(profiles_df.feat.to_list()).squeeze(1)

In [9]:
profiles_mat.shape

(4443, 512)

# Test Image

In [10]:
with open('./save_feat/feature_adaface_ir101_webface12m_test.pickle', 'rb') as f:
    test_feats = pickle.load(f)

In [11]:
len(test_feats.keys())

3728

In [12]:
np.array(list(test_feats.values())).squeeze(1).shape

(3728, 512)

In [13]:
test_face_ids = []
test_image_names = []
for k in test_feats.keys():
    test_face_ids.append(k.split('_')[0])
    test_image_names.append(k)

In [14]:
test_df = pd.DataFrame({'id':test_face_ids, 'feat':list(test_feats.values()), 'image_name':test_image_names})

In [15]:
test_df

,id,feat,image_name
0,4158,"[[-0.010510423, 0.036218934, -0.11204828, -0.0...",4158_6.jpg
1,3510,"[[-0.0020468438, -0.011027685, 0.0029530183, -...",3510_2.jpg
2,5477,"[[-0.018202154, 0.0153330825, 0.027043613, 0.0...",5477_2.jpg
3,4020,"[[0.105125435, 0.08050128, -0.05876012, 0.0515...",4020_3.jpg
4,2353,"[[0.063434534, 0.0062410254, 0.028865295, -0.0...",2353_4.jpg
...,...,...,...
3723,3190,"[[0.021140588, 0.1175212, 0.009153222, 0.04619...",3190_1.jpg
3724,159,"[[-0.0652652, 0.066973306, 0.022362689, 0.0460...",159_10.jpg
3725,3269,"[[-0.02618685, 0.08867144, 0.04528909, 0.01482...",3269_2.jpg
3726,5004,"[[0.05509101, 0.06511412, -0.079589464, -0.052...",5004_1.jpg


In [16]:
test_mat = np.array(test_df.feat.to_list()).squeeze(1)

In [17]:
test_mat.shape

(3728, 512)

# Run Compute Sims

In [18]:
sims = (np.dot(test_mat, profiles_mat.T) + 1.0) / 2.0

In [19]:
sims.shape

(3728, 4443)

In [20]:
sims

array([[0.47797447, 0.525813  , 0.47832966, ..., 0.5174275 , 0.5273663 ,
        0.5681299 ],
       [0.5533403 , 0.5754499 , 0.44753844, ..., 0.5063311 , 0.49581808,
        0.50589925],
       [0.46781805, 0.41041097, 0.53192955, ..., 0.4685638 , 0.56690764,
        0.50224984],
       ...,
       [0.54789007, 0.46486166, 0.78547394, ..., 0.6376294 , 0.49377942,
        0.4973077 ],
       [0.5648664 , 0.5702756 , 0.46885374, ..., 0.55467916, 0.47643107,
        0.51351386],
       [0.4846122 , 0.4960339 , 0.6497808 , ..., 0.7110972 , 0.5564951 ,
        0.51814353]], dtype=float32)

In [25]:
true_positive = 0
false_positive = 0
false_nagetive = 0

for i in range(sims.shape[0]):
    index = np.argmax(sims[i])
    if sims[i][index] < 0.73:
        false_nagetive += 1
    else:
        if test_df.iloc[i].id == profiles_df.iloc[index].id:
            true_positive += 1
        else:
            false_positive += 1

precision = true_positive / (true_positive + false_positive)
recall = true_positive / (true_positive + false_nagetive)
accuracy = true_positive / sims.shape[0]
print("precision: {}, recall: {}, accuracy: {}".format(precision, recall, accuracy))

precision: 0.515776188472865, recall: 0.47574699262708575, accuracy: 0.328862660944206
